# VELOCITY CALCULATION 

This notebook shows the process to compute correctly the velocity of the spheres in the 2-D vibrating experiment from the DataFrame of the original linked trajectories data of the spheres along the frames of the experiment.


### STEPS TO COMPUTE AND PROCESS DATA
1. Load df linked file 
2. Filtering points within Rlim distance to discard particles near the platform border (requirement: obtain the center of the platform (via houghcircle function) and compute r for each particle) + reindex track ids
3. Filtering tracks via savgol
4. Compute velocities vx, vy with data from timeframe (2-steps process)
5. Calculate v and add to df
6. Obtain distribution functions
7. Compute Vmean and Kinetic Energy

### Import libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pims
from pandas import DataFrame, Series  
import trackpy as tp
from scipy.signal import savgol_filter
import math
import skimage

#interactive graphics
%matplotlib widget 

from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import Image

### DEFINE FUNCTIONS

#### df_filR(rlim1,rlim2,df,frames)

The function filters the particles that are within a Rlim distance from the center of the vibrated platform. The output is a DataFrame with these particles.

INPUTS:

rlim1, rlim2: Values to determine the radius of the circular platform via skimage library function hough_radii.
frames: frames from .cine file of the experiment.

df: DataFrame with locations of the particles and linked into trajectories

OUTPUTS:

out: Dataframe with filtered positions 

In [ ]:
def df_filR(rlim1,rlim2, df, frames):
    hough_radii=np.arange(rlim1,rlim2,1);
    hough_res=skimage.transform.hough_circle(frames[100],hough_radii);
    accums,cx,cy,radii=skimage.transform.hough_circle_peaks(hough_res,hough_radii,total_num_peaks=1);
    r=np.sqrt((df.x-cx)**2+(df.y-cy)**2);
    # rlim = radii from the detected circle - 17 frames-distance, equivalent to 3.5 sphere diameter
    rlim=(radii-17.5);
    df['r']=r;
    df=df[df.r<rlim[0]];
    df.drop('r', axis=1, inplace=True)
    
    return df

#### df_filSG(df,window_length)

The function filters the particle positions to smooth trajectories and avoid failures in the location process taking into account a window length. The output is a DataFrame with these particles.

INPUTS: 

df: DataFrame with locations of the particles and linked into trajectories

window_length: Length of the trajectory that are applied to the filter.

OUTPUTS:

out: Dataframe with filtered positions 

In [2]:
def df_filSG(df, window_length):
    #df=df.rename(columns={'particle':'track'})
    window_length=window_length;
    out = pd.DataFrame()
    for trajectory in set(df.track):
        sub = df[df.track==trajectory]
    
        if sub.shape[0] <= window_length+1:
        #Para obviar los casos en los que la trayectoria dura menos que la ventana de suavizado
            pass
        else:
            printp(f'Smoothing positions for track: {trajectory : .0f}')
            # Savgol filter
            sub['x'] = savgol_filter(sub['x'], window_length=window_length, polyorder=3, axis=0)
            sub['y'] = savgol_filter(sub['y'], window_length=window_length, polyorder=3, axis=0)          
            out=pd.concat([out,sub],ignore_index=True,axis=0)
    return out

def printp(string):
    """ Modification of print function to do everything on one line """
    import sys
    sys.stdout.write('\r'+ str(string))
    sys.stdout.flush()

#### velocity(df) & veltime(df, time)

The first function calculates the particle position variation between two consecutive frames.

The second function calculates the velocity dividing the position increment by the time increment.

INPUTS:

df: DataFrame with locations of the particles and linked into trajectories

time: Time of each frame

OUTPUTS:

1. out1: Dataframe with position increments
2. out2: Dataframe with velocities vx, vy


In [ ]:
def velocity(df):
    out=pd.DataFrame()
    for i in range(ntracks):
        sub=df[df.track==i]
        vxvy = sub[['x','y']].diff(periods=-1)
        sub=sub[:-1]
        sub['vx']=vxvy.x
        sub['vy']=vxvy.y
        out=pd.concat([out,sub],ignore_index=True)
        if i%100==0:
            print(i)
    return out

def veltime(df,time):
    out=pd.DataFrame()
    for j in range(nt):
        sub=df[df.frame==j]
        vx=sub['vx']/(time[j+1]-time[j])
        vy=sub['vy']/(time[j+1]-time[j])
        sub['vx']=vx
        sub['vy']=vy
        out=pd.concat([out,sub],ignore_index=True)
        if j%100==0:
            print(j)
    return out

#### datavel (df)

The function calculates the module of the velocity for each particle in each frame and add a column to the DataFrame.

INPUTS:

df: DataFrame with locations of the particles and linked into trajectories

OUTPUTS:

df: DataFrame with velocity column

In [1]:
def datavel(df):
    v=(df.vx**2+df.vy**2)**(0.5);
    df['v']=v;
    
    return df

#### __tracks_len:__ get list of tracks ids; and no. of tracks

INPUT: data frame 'df' with tracks

OUTPUT: 
* lengths: array with lenghts of all tracks
* track_list: array with tracks IDs 
* ntracks: no. of tracks

#### __reindex_tracks:__ resets tracks IDs in logical order: 1,2,3...., ntracks

INPUT: 

* df: data frame with tracks
* arr: array with tracks list

OUTPUT: data frame with 'track' column modified

In [3]:
def tracks_len(df):
    track_list = np.unique(df.track.values) # list of current track_listk IDs
    ntracks = len(track_list)
    lengths = np.array( [len(df[df.track==i]) for i in track_list] ) # length of each track
    return track_list, lengths, ntracks


def reindex_tracks(df, arr):
    # replace tracks IDs with logical order count
    ntracks = len(arr)
    for i in range(ntracks):
        df.loc[df.track == arr[i],'track'] = i
    return df

## 1. LOAD DATA (linked df + frametime)

In [ ]:
df=pd.read_pickle('/media/juan/DiscoDuro2G/Serie_Densidad_1.8sigma/Data Processed/datalinked_densidad0.00625_f75Hz.pkl')

time=np.loadtxt('/media/juan/DiscoDuro2G/Serie_Densidad_1.8sigma/Data Processed/frametime_densidad0.00625_f75Hz.txt')

nt = np.max(df.frame.values) # number of frames (assumming there is at least one complete track)

df=df.rename(columns={'particle':'track'})

In [ ]:
track_list, lengths, ntracks = tracks_len(df)
#reindex se haría después de filtrar
df2=reindex_tracks(df, track_list)